# TODO

QOL
- Make sum work directly on Private Syft Tensors, instead of having to call it on Tensor.child

In [19]:
%load_ext autoreload
%%autoreload

UsageError: Line magic function `%%autoreload` not found.


In [1]:
import syft as sy
import numpy as np
from syft.core.adp.entity import Entity

In [2]:
person1 = Entity(name="1")
person2 = Entity(name="2")

entities = np.random.choice([person1, person2], size=10**6)

In [3]:
tensor = sy.Tensor(np.ones(10**6, dtype=np.int32)).private(min_val=0, max_val=2, entities=entities)

In [4]:
from syft.core.tensor.autodp.ndim_entity_phi import NDimEntityPhiTensor as NDEPT
assert isinstance(tensor.child, NDEPT), f"Please double check that 'ndept' is set to True in the _private() method in the ancestors.py file"

In [13]:
result = tensor.sum()

Exception: Data: GammaTensor(value=1000000, data_subjects=<syft.core.adp.entity_list.EntityList object at 0x7f552d223430>, min_val=0, max_val=2000000, func=<function no_op at 0x7f553e782e50>, id='3198124266', state={'3198124266': ...}) ,type: <class 'syft.core.tensor.autodp.gamma_tensor.GammaTensor'> must be list or nd.array 

In [5]:
result = tensor.child.sum()

In [6]:
assert result.value == 10**6
assert result.max_val/10**6 == 2

In [7]:
from nacl.signing import VerifyKey, SigningKey

key = SigningKey.generate()

vk = key.verify_key

In [8]:
from syft.core.adp.adversarial_accountant import AdversarialAccountant

acc = AdversarialAccountant()

In [9]:
result.publish(accountant=acc, sigma=200_000, user_key=vk)

1000000000000
4000000000000


DeviceArray(1, dtype=int32)

In [12]:
from jax import numpy as jnp

In [13]:
result.max_val

2000000

In [14]:
result.min_val

0

In [15]:
result.max_val - result.min_val

2000000

In [17]:
jnp.sum(jnp.square(result.max_val - result.min_val))

DeviceArray(1385447424, dtype=int32)

In [19]:
jnp.sum(jnp.square(result.value))

DeviceArray(-727379968, dtype=int32)

In [22]:
jnp.square(result.value)  # ERROR!!

DeviceArray(-727379968, dtype=int32)

In [24]:
jnp.sqrt(jnp.sum(jnp.square(result.value)))

DeviceArray(nan, dtype=float32)

In [23]:
result.value.mean()

1000000.0

In [26]:
jnp.sqrt(jnp.sum(result.value))

DeviceArray(1000., dtype=float32)

In [25]:
import numpy as np
np.square(result.value)

1000000000000

In [27]:
%%timeit
np.sqrt(np.sum(np.square(result.value)))

7.4 µs ± 155 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [28]:
%%timeit
jnp.sqrt(jnp.sum(jnp.square(result.value)))

11.2 µs ± 113 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [29]:
%%time
np.sqrt(np.sum(np.square(result.value)))

CPU times: user 147 µs, sys: 5 µs, total: 152 µs
Wall time: 168 µs


1000000.0

In [30]:
%%time
jnp.sqrt(jnp.sum(jnp.square(result.value)))

CPU times: user 479 µs, sys: 17 µs, total: 496 µs
Wall time: 352 µs


DeviceArray(nan, dtype=float32)

In [31]:
%%time
np.sum(np.square(result.value))

CPU times: user 134 µs, sys: 5 µs, total: 139 µs
Wall time: 156 µs


1000000000000

In [10]:
%%time
np.ones_like(result.value)

CPU times: user 41 µs, sys: 3 µs, total: 44 µs
Wall time: 52 µs


array(1)

In [13]:
%%time
jnp.ones_like(result.value)

CPU times: user 471 µs, sys: 41 µs, total: 512 µs
Wall time: 526 µs


DeviceArray(1, dtype=int32)